Textbook RSA Dummy Implementation
===================

This file is an implementation of the scheme invented by Rivest, Shamir and Adleman.

Recall the __RSA experiment__: 

Given security parameter $\lambda = 1^n$ a PPT adversary $\mathcal{A}$ and PPT algorithm $\texttt{GenRSA}$, the RSA experiment is as follows
1. Run $(N, e, d) \leftarrow \texttt{GenRSA}(\lambda)$.
2. Uniformly pick $y \leftarrow \mathbb{Z}^*_N$.
3. Run $x = \mathcal{A}(N, e, y)$ where $x \in \mathbb{Z}^*_N$.
4. Set $\texttt{Expr}^{\text{RSA}}_{\mathcal{A}, \texttt{GenRSA}}(\lambda) = 1$ if $x^e = y$ and $0$ otherwise.

Informally, the RSA experiment is merely asking the adversary to take the $e$th square root of $y$ modulo $N$. Now, for the hardness assumption:

__RSA hardness assumption__:

The RSA experiment is hard relative to $\texttt{GenRSA}$ is every PPT adversary $\mathcal{A}$ has negligible advantage of succeeding to solve the experiment:
$$
\texttt{Adv}^{\text{RSA}}_{\mathcal{A}, \texttt{GenRSA}}(\lambda) = \mathbb{P}[\texttt{Expr}^{\text{RSA}}_{\mathcal{A}, \texttt{GenRSA}}(\lambda) = 1] \leq \text{negl}(n).
$$

Recall, that it is conjectured that if $N = pq$ for some $p, q = \mathcal{O}(\sqrt{N})$ primes and arbitrary exponent $e$ the RSA experiment is indeed very hard to solve (it is NP-hard). We can then use this to create a trapdoor and based on that a crypto scheme, that we will see below.

In [ ]:
load('rsa.sage')

Key Generation
-----------------------------------------

We begin with key generation. Given a security parameter $\lambda = 1^n$ we perform the following:

1. Pick two different primes $|p| = |q|$ (here $|\cdot|$ measures the length of the binary representation of numbers).
2. Calculate $N = pq$ and $\phi(N) = (p - 1)(q - 1)$.
3. Pick an element $e$ such that $\gcd(e, \phi(N)) = 1$. This means that $e \in \mathbb{Z}^*_N$, and hence invertible mod $N$.
4. Output $pk = (N, e)$ and $sk = (N, e^{-1})$

Below we have performed this procedure precisely:

In [2]:
pk, sk = key_gen(50);pk, sk

((2457322981641903598394904537989, 552540427338418501877724158197),
 (2457322981641903598394904537989, -207543256224699247221522360227))

Encryption
---------------------------------

Encryption is rather simple, we are just following the RSA experiment:

Given public key $pk = (N, e)$ and message $m$, output $c = m^e \mod N$.



In [3]:
m = 1234
c = encrypt(pk, m);c

1258967897177224801149888061088

Decryption
---------------------------------------

Decryption will make use of our trapdoor: when we are only given $N, e$ and $x^e$ for some $x$, it is hard to invert the encryption function. However, leveraging the fact that $e$ is invertible in $\mathbb{Z}^*_N$ and in any group $G$ we have $g^p = g^{p \mod |G|}$ for arbitrary $p \in \mathbb{Z}$, given a secret key $sk = (N, d = e^{-1})$ and ciphertext $c$, we can calculate and output
$$
    m' = c^d\mod N.
$$

Correctness
----------------------------------------

Combining the encryption and decryption functions then, given $pk = (N, e)$, $sk = (N, d = e^{-1})$ and message $m \in \mathbb{Z}_N$, we have


$$ 
    m' = \text{Dec}_{sk}(\text{Enc}_{pk}(m))
$$


$$
    m' = (m^e\mod N)^d\mod N
$$

$$
    m' = m^{ed\mod N}\mod N
$$
Then, $ed = ee^{-1} = 1 \mod N$ by the definition of the cipher and hence
$$
    m' = m^{1 \mod N}\mod N = m.
$$

Below, we see can test the correctness in practice:

In [4]:


m_prime = decrypt(sk, c);m_prime

1234